<a href="https://colab.research.google.com/github/TankMermaid/spike-in/blob/master/funGene.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!/usr/bin/python3


__Author__= 'Tank'
__Email__ = 'xnzhang@genetics.ac.cn'
__Version__=1.0


"""This script aims to scrape some website table-format \
data and apply in analysing some basic statistics and visualization"""
from google.colab import drive


import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import json
import os
import sys
import subprocess

drive.mount('/content/gdrive')

url="http://fungene.cme.msu.edu/"
# new_dic = {}


def get_span_name(table_body):
  span_names=[]
  for tb in table_body[1:]:
    # print(type(tb))
    rows = tb.find_all('td',{"style": True} )
    print("here is a tr block td data")
    # print(rows)
    print(len(rows))
    for r in rows:
      span=r.find_all('span',{"class":"hilite"})
      span_name="".join([y.text.strip() for y in span])
      # print(span_name)
      # print(type(span_name))
      if len(span_name) >= 1:
        span_names.append(span_name)
        # print(span_names)
  return span_names

def get_table_number(table_body):
  for tb in table_body:
    print("What you are scrapying is a table...")
    # print(tb)
    print("all the table number is %d"%(len(table_body)))

def get_gene_url_list(tds):
  ## parallel in multiple colab running 
  for td in tds:
    # span=td.find_all('span',{"class":"gene"})
    span=td.find_all('span')
    # span_name=[y.text.strip() for y in span]
    # print(span)# 
    td_name=td.find('span',{"class":"hilite"}).text
    # print(td_name)
    cwd=os.getcwd()
    try:
      os.makedirs(os.path.join(cwd,"gdrive/My Drive/",td_name),exist_ok=True) ## mkdir and mkdirs and makedirs ## join and os.path.join
    except:
      pass

    link_list=[]
    gene_list=[]
    for href in span:
      link = href.find_all('a')
      links="".join([url + x.get('href') for x in link])
      # print(links)
      if len(links) >0:
        link_list.append(links)

      genes="".join([y.text.strip() for y in link])
      if len(genes) >0:
        gene_list.append(genes)
    # print(link_list)
    # print(gene_list)
    # print(len(gene_list))
    # print(len(link_list))
  return link_list, gene_list

def get_page_number(gene_list,link_list):
  gene_url_map = dict(zip(gene_list,link_list))  ## test
  print(gene_url_map)
  for k,v in gene_url_map.items():
    print(k)
    print(v)
    response_sub= requests.get(v)
    html_sub=response_sub.content.decode("gbk")
    bs_sub = BeautifulSoup(html_sub, "html.parser")
    # rows_sub = [tag for tag in bs_sub.find_all('td') if tag.text == "Page"]
    rows_sub = bs_sub.find_all('td')
    # td_page= re.findall(r'<td>Page:.*?</td>',rows_sub)
    for i in rows_sub:
      # a=[x.find('a') for x in i if x.find('a') is not None]
      page_a_hit =i.find_all("a")
      
      page_href=[x.get('href')  for x in page_a_hit if x.get('href').startswith("?hmm_id=")]
      if len(page_href) >0:
        print(page_href)
        page_length=len(p_href)+1
        print("scrapying page number is %d" %(page_length))
    # print("here is a page")
    # print(td_page)


def get_data(url):
  response= requests.get(url)
  html=response.content.decode("gbk")
    
  bs=BeautifulSoup(html, "html.parser")
  table_body=bs.find("table")
  # print(table_body)
  # table_body=bs.find_all("table")
  ## get table number
  get_table_number(table_body)

  # get span name
  # span_names=get_span_name(table_body)

   ## get td list
  # td_list=[]
  # for tb in table_body[1::]:
  tb = table_body
    # print(type(tb))
  tds = tb.find_all('td',{"style": True} )
  print("here is a tr block td data")
  # print(rows)
  print("all the table section number is %d" %(len(tds)))

  # link_list,gene_list = get_gene_url_list(tds)
  for td in tds[0:2]:
    new_dic={}
    # span=td.find_all('span',{"class":"gene"})
    span=td.find_all('span')
    # span_name=[y.text.strip() for y in span]
    # print(span)# 
    td_name=td.find('span',{"class":"hilite"}).text
    # print(td_name)
    cwd=os.getcwd()
    try:
      os.makedirs(os.path.join(cwd,"gdrive/My Drive/",td_name),exist_ok=True) ## mkdir and mkdirs and makedirs ## join and os.path.join
    except:
      pass

    link_list=[]
    gene_list=[]
    for href in span:
      link = href.find_all('a')
      links="".join([url + x.get('href') for x in link])
      # print(links)
      if len(links) >0:
        link_list.append(links)

      genes="".join([y.text.strip() for y in link])
      if len(genes) >0:
        gene_list.append(genes) 
    
    
    # get_next_layer(gene_list,link_list)
    gene_url_map = dict(zip(gene_list,link_list))  ## test
    print(gene_url_map)
    for k,v in gene_url_map.items():
      print(k)
      print(v)
      response_sub= requests.get(v)
      html_sub=response_sub.content.decode("gbk")
      bs_sub = BeautifulSoup(html_sub, "html.parser")
      # rows_sub = [tag for tag in bs_sub.find_all('td') if tag.text == "Page"]
      rows_sub = bs_sub.find_all('td')
      # td_page= re.findall(r'<td>Page:.*?</td>',rows_sub)
      pageN=0 ##init
      for i in rows_sub:
        # a=[x.find('a') for x in i if x.find('a') is not None]
        page_a_hit =i.find_all("a")
        
        page_href=[x.get('href')  for x in page_a_hit if x.get('href').startswith("?hmm_id=")]
        if len(page_href) >0:
          print(page_href)
          page_length=len(page_href)+1
          print("scrapying page number is %d" %(page_length))
          pageN=page_length
      
      

      page_gene_link=[] 
      for pg in range(1,pageN): 
        for i in rows_sub:
          temp_dict = {}
          page_a_hit =i.find_all("a")
          gene_prot_href="".join([url + x.get('href')+ "&page="+ str(pg) for x in page_a_hit if x.get('href').startswith("gpprotdata")])
          gene_prot_name="".join([y.text.strip() for y in page_a_hit])

          # print(gene_prot_href)
          # print(gene_prot_name)
          if len(gene_prot_href) >0 and len(gene_prot_name)>0:
            print(gene_prot_href)
            page_gene_link.append(gene_prot_href)
            
            response_sub_sub= requests.get(gene_prot_href)
            html_sub_sub=response_sub_sub.content.decode("gbk")
            bs_sub_sub = BeautifulSoup(html_sub_sub, "html.parser")
            # print(bs_sub_sub)
            rows_sub_sub = bs_sub_sub.find_all('a')
            rows_sub_sub=rows_sub_sub[1:]
            # print(rows_sub_sub)
            ncbi_href=[x.get('href')  for x in rows_sub_sub if x.get('href').startswith("http://www.ncbi")]
            # ncbi_name=[y.text.strip() for y in rows_sub_sub]
            print(ncbi_href[0])
            # print(ncbi_name)
            k_s = td_name + "-" + k  + "-" + gene_prot_name + "-" + str(pg)
            print(k_s)
            print(ncbi_href[0])
            temp_dict[k_s] = ncbi_href[0]
            new_dic[k_s]=ncbi_href[0] ## 0 fasta 1 genebank
            cwd=os.getcwd()
            print(td_name)
            print(cwd)
            try:
              dir_path = os.path.join(cwd,"gdrive/My Drive",td_name,k,gene_prot_name,str(pg))
              os.makedirs(dir_path,exist_ok=True) ## mkdir and mkdirs and makedirs ## join and os.path.join
              jsObj = json.dumps(temp_dict, indent=4)
              fh = open(os.path.join(dir_path,'fungene_url.json'), 'w') 
              fh.write(jsObj) 
              fh.close()
            except:
              pass
            # print(gene_prot_name)
        # if len(page_gene_link) >0:
          # print(page_gene_link)

    try:
      dir_path = os.path.join(cwd,"gdrive/My Drive",td_name,k,"summry")
      os.makedirs(dir_path,exist_ok=True) ## mkdir and mkdirs and makedirs ## join and os.path.join
      jsObj = json.dumps(new_dict, indent=4)
      fh = open((k+'fungene_url.json'), 'w') 
      fh.write(jsObj) 
      fh.close()
    except:
      pass



        # for lk in page_gene_link:
        #   response_sub_sub= requests.get(lk)
        #     html_sub_sub=response_sub_sub.content.decode("gbk")
        #     bs_sub_sub = BeautifulSoup(html_sub_sub, "html.parser")
        #     # print(bs_sub_sub)
        #     rows_sub_sub = bs_sub_sub.find_all('a')
        #     rows_sub_sub=rows_sub_sub[1:]
        #     # print(rows_sub_sub)
        #     ncbi_href=[x.get('href')  for x in rows_sub_sub if x.get('href').startswith("http://www.ncbi")]
        #     ncbi_name=[y.text.strip() for y in rows_sub_sub]
        #     # print(ncbi_href[1])
        #     # print(ncbi_name)
        #     k_s = k + "-" + k1
        #     print(k_s)
        #     print(ncbi_href[0])
        #     new_dic[k_s]=ncbi_href[0] ## 0 fasta 1 genebank
                



        
          # for v1 in c_sub.items():
          #       # print(v1)
          #       response_sub_sub= requests.get(v1)
          #       html_sub_sub=response_sub_sub.content.decode("gbk")
          #       bs_sub_sub = BeautifulSoup(html_sub_sub, "html.parser")
          #       # print(bs_sub_sub)
          #       rows_sub_sub = bs_sub_sub.find_all('a')
          #       rows_sub_sub=rows_sub_sub[1:]
          #       # print(rows_sub_sub)
          #       ncbi_href=[x.get('href')  for x in rows_sub_sub if x.get('href').startswith("http://www.ncbi")]
          #       ncbi_name=[y.text.strip() for y in rows_sub_sub]
          #       # print(ncbi_href[1])
          #       # print(ncbi_name)
          #       k_s = k + "-" + k1
          #       print(k_s)
          #       print(ncbi_href[0])
          #       new_dic[k_s]=ncbi_href[0] ## 0 fasta 1 genebank
                


        
        

      

        
    # print(rows_sub)

        # p_names=[]
        # p_hrefs=[]
        # for line in rows_sub: ## test
    # for line in rows_sub:
          # print("hello")
  

  # ## get td list
  # td_list=[]
  # # for tb in table_body[1::]:
  # for tb in table_body:
  #   # print(type(tb))
  #   tds = tb.find_all('td',{"style": True} )
  #   print("here is a tr block td data")
  #   # print(rows)
  #   print(len(tds))
   
  #   for td in tds:
  #     span=td.find_all('span',{"class":"gene"})
  #     # span_name=[y.text.strip() for y in span]
  #     print(span)# 


  #   # rows = table_body[1].find_all('td')
  #   # print(rows)
  #   # print(len(rows))

  #   # names=[]
  #   # hrefs=[]
  #   # row and column handle
  #   for row in rows[3:4]:  ## td3
  #     cols=row.find_all('a')
  #     href=[url + x.get('href') for x in cols if x.get('href') is not None ]
  #     # conc_href=url+href
  #     name=[y.text.strip() for y in cols]
  #     # cols=[x.text.strip() for x in cols]
  #     print(len(href))
  #     print(len(name))
  #     # names.append(name)
  #     # hrefs.append(href)
  #     # c=dict(zip(name,href))
  #     c=dict(zip(name,href))  ## test
  #     # print(hrefs)
  #     # print(names)
  #     # print(list(c))
  #     # print(c)
  #     for k,v in c.items():
  #       print(k)
  #       print(v)
  #       response_sub= requests.get(v)
  #       html_sub=response_sub.content.decode("gbk")
  #       bs_sub = BeautifulSoup(html_sub, "html.parser")
  #       # print(bs_sub)
  #       # table_body_sub = bs_sub.find('tr')
  #       rows_sub = bs_sub.find_all('td')
  #       # print(rows_sub)

  #       # p_names=[]
  #       # p_hrefs=[]
  #       # for line in rows_sub: ## test
  #       for line in rows_sub:
  #         # print("hello")
  #         # print(len(rows_sub))
  #         # print("hi")
  #         cols_sub=line.find_all('a')
  #         # print("hello")
  #         # print(cols_sub)
  #         # print("hi")
  #         if len(cols_sub) == 1:
  #           p_href=[url + x.get('href')  for x in cols_sub if x.get('href').startswith("gpprotdata")]
  #           p_name=[y.text.strip() for y in cols_sub]
  #           # print("hello")
  #           # print(p_href)
  #           # print(p_name)
  #           if len(p_href) != 0:
  #             c_sub=dict(zip(p_name,p_href))
  #             # print(c_sub)
              
  #             for k1,v1 in c_sub.items():
  #               # print(v1)
  #               response_sub_sub= requests.get(v1)
  #               html_sub_sub=response_sub_sub.content.decode("gbk")
  #               bs_sub_sub = BeautifulSoup(html_sub_sub, "html.parser")
  #               # print(bs_sub_sub)
  #               rows_sub_sub = bs_sub_sub.find_all('a')
  #               rows_sub_sub=rows_sub_sub[1:]
  #               # print(rows_sub_sub)
  #               ncbi_href=[x.get('href')  for x in rows_sub_sub if x.get('href').startswith("http://www.ncbi")]
  #               ncbi_name=[y.text.strip() for y in rows_sub_sub]
  #               # print(ncbi_href[1])
  #               # print(ncbi_name)
  #               k_s = k + "-" + k1
  #               print(k_s)
  #               print(ncbi_href[0])
  #               new_dic[k_s]=ncbi_href[0] ## 0 fasta 1 genebank
                



  #         else:
  #           pass

  #   return new_dic

def main():
  get_data(url)  

  # jsObj = json.dumps(dictObj, indent=4)
  # fileObject = open('./fungene.json', 'w') 
  # fileObject.write(jsObj) 
  # fileObject.close()

if __name__ == '__main__':
    main()
    # print(__name__)
     
  
    

In [ ]:
os.makedirs

In [ ]:
import os
# print(os.path)
os.getcwd()

'/content'

In [ ]:
!ls -l drive/My\ Drive

In [ ]:
# js=pd.read_json("./fungene.json",encoding='utf-8',orient= "index")

# type(js)

with open("./fungene.json") as f:
  js = json.load(f)
  print(js)
  # t=type(js)
  # print(t)
  for k, v in js.items():
    cmd = ["wget",v,"-O", '{:s}/{:s}.fasta'.format(k,k)]
    print(cmd)
    subprocess.run(cmd)
  f.close()


In [ ]:
!python drive/My\ Drive/funGeneDld.py 2>&1 >> drive/My\ Drive/dld1.log